In [2]:
import numpy as np
import os
import helpers
import pickle
from matplotlib import pyplot as plt
from scipy import signal
from sklearn import preprocessing


In [3]:
labels = helpers.load_data("trainLabels.npy", "train")

#Smartglasses
jins_accel = helpers.load_data("trainJinsAccelerometer.npy", "train")
jins_gyro = helpers.load_data("trainJinsGyroscope.npy", "train")

#Smartwatch
ms_accel = helpers.load_data("trainMSAccelerometer.npy", "train")
ms_gyro = helpers.load_data("trainMSGyroscope.npy", "train")

#Smartphone
nx_gravity = helpers.load_data("trainGravity.npy", "train")
nx_magnet = helpers.load_data("trainMagnetometer.npy", "train")
nx_linAcc = helpers.load_data("trainLinearAcceleration.npy", "train")
nx_accel = helpers.load_data("trainAccelerometer.npy", "train")
nx_gyro = helpers.load_data("trainGyroscope.npy", "train")

In [5]:
jins_accel = helpers.preprocess_data(jins_accel, sampling_type="upsample")
jins_gyro = helpers.preprocess_data(jins_gyro, "upsample")
ms_accel = helpers.preprocess_data(ms_accel, "upsample") #actually getting downsampled, but using resample instead of decimates
ms_gyro = helpers.preprocess_data(ms_gyro)
nx_gravity = helpers.preprocess_data(nx_gravity)
nx_magnet = helpers.preprocess_data(nx_magnet) #not getting resampled, but still needs other preprocessing steps
nx_linAcc = helpers.preprocess_data(nx_linAcc)
nx_accel = helpers.preprocess_data(nx_accel)
nx_gyro = helpers.preprocess_data(nx_gyro)


/home/wph/.local/lib/python3.8/site-packages/scipy/signal/_signaltools.py:1611: RuntimeWarning: divide by zero encountered in true_divide
  res *= (1 - noise / lVar)
/home/wph/.local/lib/python3.8/site-packages/scipy/signal/_signaltools.py:1611: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)
/home/wph/.local/lib/python3.8/site-packages/scipy/signal/_signaltools.py:1611: RuntimeWarning: divide by zero encountered in true_divide
  res *= (1 - noise / lVar)
/home/wph/.local/lib/python3.8/site-packages/scipy/signal/_signaltools.py:1611: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/wph/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2735/3494985010.py", line 4, in <cell line: 4>
    ms_gyro = helpers.preprocess_data(ms_gyro)
  File "/home/wph/Documents/CogActivity/helpers.py", line 50, in preprocess_data
    out_data = downsample(data, target_size)
  File "/home/wph/Documents/CogActivity/helpers.py", line 20, in downsample
    downsampled_data[i,:,j] = signal.decimate(data[i,:,j], downsample_factor)
ValueError: could not broadcast input array from shape (268,) into shape (200,)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wph/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/wph/.local/lib/python3.8/site-package